In [1]:
import json, numpy as np, pandas as pd, matplotlib.pyplot as plt
import time
from aukf  import UnscentedKalman           # CV-UKF
from utils import OrbitPropagator           # 2-body propagator

plt.rcParams["figure.dpi"] = 110

def rms(a, b):
    return np.sqrt(np.mean((a-b)**2))


File or folder: C:\Users\nazih\satellite-aukf-assignment\orekit-data.zip  not found


            The Orekit library relies on some external data for physical models.
            Typical data are the Earth Orientation Parameters and the leap seconds history,
            both being provided by the IERS or the planetary ephemerides provided by JPL.
            Such data is stored in text or binary files with specific formats that Orekit knows
            how to read, and needs to be provided for the library to work.

            You can download a starting file with this data from the orekit gitlab at:
            https://gitlab.orekit.org/orekit/orekit-data

            or by the function:
            orekit.pyhelpers.download_orekit_data_curdir()

            
✔️  Orekit data registered: C:\Users\nazih\satellite-aukf-assignment\orekit-data


In [ ]:
df         = pd.read_parquet("GPS_clean.parquet")
meas_cols  = json.load(open("meas_cols.json"))
epochs     = df["time"].values.astype("datetime64[s]")
Z          = df[meas_cols].values

# a high-fidelity “truth” orbit propagated with Orekit
prop_truth = OrbitPropagator()                      # 8×8 gravity, tides, drag …
tspan, sts = prop_truth.propagate(
                prop_truth._fake_init_orbit(),
                duration=(epochs[-1]-epochs[0]).astype(int).item(),  # .item() → float
                step=1.0)

print(f"Propagate took {time.perf_counter()-t0:.1f} s, {len(sts)} steps")

truth_xyz  = np.array([[s.getPVCoordinates().getPosition().getX(),
                        s.getPVCoordinates().getPosition().getY(),
                        s.getPVCoordinates().getPosition().getZ()] for s in sts])


In [ ]:
# --- constant-velocity UKF ------------------------------
ukf_cv = UnscentedKalman(meas_cols, q0=1e-2, r0=25.0, adaptive="iae")
ukf_cv.init_from_measurement(0.0, Z[0])

# --- two-body (Orekit) UKF ------------------------------
prop_2b      = OrbitPropagator()      # lean 2×2 gravity only
ukf_2b       = UnscentedKalman(meas_cols, propagator=prop_2b,
                               q0=1e-2, r0=25.0)
ukf_2b.init_from_measurement(0.0, Z[0])

prev_t = epochs[0]
for t, z in zip(epochs[1:], Z[1:]):
    dt = float((t - prev_t).astype(int))
    ukf_cv.step(dt, z)
    ukf_2b.step(dt, z)
    prev_t = t

cv_xyz = np.vstack([x[1][:3] for x in ukf_cv.history])
tb_xyz = np.vstack([x[1][:3] for x in ukf_2b.history])


In [ ]:
def show_track(xyz, label, c):
    plt.plot(xyz[:,0]/1e3, xyz[:,1]/1e3, lw=.6, label=label, c=c)

plt.figure(figsize=(5,5))
show_track(truth_xyz, "truth",   "k")
show_track(cv_xyz,     "CV-UKF", "#1f77b4")
show_track(tb_xyz,     "2-body-UKF", "#ff7f0e")
plt.gca().set_aspect("equal"); plt.legend(); plt.title("XY ground-track")
plt.tight_layout()
plt.show()

print("RMS position error  [m]")
print(f"  CV  : {rms(cv_xyz, truth_xyz):.1f}")
print(f"  2-b : {rms(tb_xyz, truth_xyz):.1f}")
